In [ ]:
import folium
import data.DownloadData as DD
import data.DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [ ]:
# récupération des contours départementaux
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson') as f:
    departements = geojson.load(f)

In [ ]:
# récupération des données sur les stations-service
DD.get_ten_Mins_Data()
DD.get_dailyData()

# on récupère le J-5 pour observer la variation 
chaine = DD.get_chaine()

# calcul des prix moyen pour les départements
dep_prices = DS.DictDepartements("PrixCarburants_instantane.xml")
dep_prices_B = DS.DictDepartements("PrixCarburants_quotidien_" + chaine + ".xml")

# calcul des variations
variations_dep = {key: dep_prices[key] - dep_prices_B.get(key, 0) for key in dep_prices.keys()}

In [ ]:
# tendance départementale
average_dep = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=departements,
    name="choropleth",
    data=dep_prices,
    key_on="feature.properties.code",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France departements", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_dep)

folium.LayerControl().add_to(average_dep)

average_dep

In [ ]:
# ajout des flèches de variations
for i in range (0, len(departements["features"])):
    if departements["features"][i].geometry.type == 'Polygon':
        box_str = str(departements["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        loc = [center['coordinates'][1], center['coordinates'][0]]
        code_p = departements["features"][i]["properties"]["code"]
        if code_p in variations_dep:
            pop = f'CP:{code_p}</br>{round(variations_dep[code_p], 4)}'
            if variations_dep[code_p] < 0:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/down.png',icon_size=(12,12))).add_to(average_dep)
            elif 0 < variations_dep[code_p] <= max(variations_dep.values())/1.5:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_1.png',icon_size=(12,12))).add_to(average_dep)
            elif max(variations_dep.values())/1.5 < variations_dep[code_p] <= max(variations_dep.values()):
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_2.png',icon_size=(12,12))).add_to(average_dep)
            else:
                folium.Marker(location = loc,
                                  popup = f'CP:{code_p}',
                                  icon = folium.features.CustomIcon('icon/equal.png',icon_size=(16,16))).add_to(average_dep)
                
average_dep